In [120]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import KFold,StratifiedKFold

In [121]:
train_df = pd.read_csv('input/train.csv')
test_df = pd.read_csv('input/test.csv')

In [122]:
ADD_FEATURES = True
USE_STACKING = False
WITH_ORG_FEATURES = True

X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [123]:
X.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [124]:
sc = StandardScaler()
sc.fit(X)
X = pd.DataFrame(sc.transform(X), columns=X.columns.values)
X_test = pd.DataFrame(sc.transform(X), columns=X_test.columns.values)

In [125]:
X.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,-0.577102,-1.273737,0.451707,-0.833709,0.235571,-0.536430,-0.334926,0.608751,-1.561580,-1.473796,...,0.263374,-1.149158,0.817469,-0.411013,0.168705,-1.578117,1.022131,-0.373968,-1.026398,0.214135
1,0.269959,-0.622138,1.190360,-0.688846,0.790975,1.539900,0.244461,-0.003525,0.858974,0.419300,...,0.966611,0.093605,0.443623,1.908764,-0.817594,1.522342,1.067654,-0.129400,0.825417,0.505685
2,-0.681113,-0.276066,0.516988,0.536516,-0.305477,-0.511033,1.769839,-0.564749,-1.561370,-1.307408,...,-0.072093,0.777997,-0.174131,-0.412316,1.151591,2.297370,-1.617906,-0.695141,-0.381449,0.356681
3,0.125158,-0.129426,-0.667575,0.195355,0.927992,0.410672,0.500633,-0.474201,-1.843910,0.548767,...,0.270216,-0.891456,-0.818468,-0.478548,1.607869,-0.789517,-0.959020,1.501744,0.697118,-0.543502
4,-0.277303,0.035610,0.817683,-0.077829,0.738607,0.955574,0.613372,0.791544,1.794753,0.090006,...,-1.036191,0.688988,-1.405987,1.468536,-1.501101,-0.958473,0.297627,0.645537,0.706318,-0.525375


In [126]:
X_test.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,-3.702913,0.087378,-3.886377,-3.734247,-6.680092,0.575956,-6.628019,-4.662606,-0.553840,-7.320278,...,-0.651562,-2.840495,-0.751052,-0.937567,-5.685551,-1.004659,-0.234896,-10.071498,-5.611912,0.339210
1,-3.424279,0.248266,-3.606678,-3.663351,-6.337915,0.840011,-5.959447,-4.841735,0.172480,-5.787489,...,-0.497341,-2.429428,-1.003920,-0.356463,-6.000145,1.164452,-0.226549,-9.806131,-4.996882,0.367142
2,-3.737127,0.333715,-3.861658,-3.063657,-7.013426,0.579186,-4.199271,-5.005928,-0.553777,-7.185558,...,-0.725131,-2.203053,-1.421768,-0.937893,-5.372046,1.706668,-0.718919,-10.419983,-5.397710,0.352867
3,-3.471910,0.369922,-4.310205,-3.230622,-6.253500,0.696403,-5.663843,-4.979437,-0.638557,-5.682664,...,-0.650062,-2.755255,-1.857596,-0.954485,-5.226510,-0.452947,-0.598119,-8.036271,-5.039493,0.266626
4,-3.604297,0.410671,-3.747796,-3.364318,-6.370178,0.765700,-5.533751,-4.609127,0.453274,-6.054110,...,-0.936560,-2.232494,-2.254994,-0.466740,-6.218159,-0.571150,-0.367726,-8.965291,-5.036438,0.268362


In [127]:
def add_feature(o, r, f, s, t):
    if s == t:
        return
    if s > t:
        (s, t) = (t, s)
    name = f + "_" + str(s) + "_" + str(t)
    if f == "mean":
        o[name] = r.iloc[:,[s,t]].mean(axis=1)
    if f == "max":
        o[name] = r.iloc[:,[s,t]].max(axis=1)
    if f == "min":
        o[name] = r.iloc[:,[s,t]].min(axis=1)
    if f == "std":
        o[name] = r.iloc[:,[s,t]].std(axis=1)
    if f == "mean_dev_std":
        o[name] = r.iloc[:,[s,t]].mean(axis=1) / r.iloc[:,[s,t]].std(axis=1)

def add_features_both(f, s, t):
    add_feature(test_df, X_test, f, s, t)
    add_feature(train_df, X, f, s, t)

def add_features_all(s, t):
    if t >= 199:
        t = 199
    #add_features_both("mean", s, t)
    #add_features_both("max", s, t)
    #add_features_both("min", s, t)
    #add_features_both("std", s, t)
    add_features_both("mean_dev_std", s, t)

In [128]:
if ADD_FEATURES:
    add_features_all(0, 199)
    for i in range(0,191,5):
        add_features_all(i, i + 4)

In [129]:
test_df.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,mean_dev_std_145_149,mean_dev_std_150_154,mean_dev_std_155_159,mean_dev_std_160_164,mean_dev_std_165_169,mean_dev_std_170_174,mean_dev_std_175_179,mean_dev_std_180_184,mean_dev_std_185_189,mean_dev_std_190_194
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-1.608149,-1.045347,-0.475090,-0.327048,-1.182033,-0.837609,-1.131746,-0.415472,0.112337,-0.890152
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,-4.371639,-1.246999,-0.497493,-0.463703,-1.271714,-0.569389,-0.655675,-0.353988,-0.289971,-0.834923
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-1.856973,-1.210591,-0.488297,-0.421577,-1.219332,-0.923047,-1.132208,-0.323641,-0.362106,-0.927789
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,-1.621730,-0.958179,-0.653155,-0.560528,-1.119150,-0.907030,-0.898232,-0.365489,1.593851,-0.907989
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,-2.053970,-1.150454,-0.394181,-0.308844,-1.147255,-0.670856,-1.001080,-0.426009,-0.331024,-0.957882


In [130]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,mean_dev_std_145_149,mean_dev_std_150_154,mean_dev_std_155_159,mean_dev_std_160_164,mean_dev_std_165_169,mean_dev_std_170_174,mean_dev_std_175_179,mean_dev_std_180_184,mean_dev_std_185_189,mean_dev_std_190_194
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.819083,57.147937,2.092287,0.259931,-1.321391,-1.093963,-1.131768,0.093384,0.603906,3.227312
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,0.184871,0.006106,1.176163,0.071424,-0.215371,12.056620,0.543880,0.225690,-0.985959,0.059058
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,-1.045980,-1.697118,4.508609,-0.356016,3.450665,-0.249342,-1.237436,0.409904,-0.999553,0.623788
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,-2.608053,1.194680,-0.346637,-0.257923,-2.229138,-1.980090,-0.034716,-0.929145,4.669675,0.992789
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,1.655779,0.348970,0.437745,2.072087,0.183004,0.104375,2.574289,-0.870095,-5.874685,-3.859106


In [131]:
train_df.cov()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,mean_dev_std_145_149,mean_dev_std_150_154,mean_dev_std_155_159,mean_dev_std_160_164,mean_dev_std_165_169,mean_dev_std_170_174,mean_dev_std_175_179,mean_dev_std_180_184,mean_dev_std_185_189,mean_dev_std_190_194
target,0.090392,0.047884,0.061300,0.044361,0.006792,0.005326,0.073237,0.017387,-0.003108,0.019623,...,-3.755446e+00,1.741075e-01,2.018483e+00,-0.077990,-0.165118,-0.083439,4.113779e-01,-0.132065,6.244625e-01,0.011733
var_0,0.047884,9.241909,-0.006694,0.052770,0.023610,0.006542,0.072820,0.018396,0.025240,0.050276,...,2.925829e+00,4.940959e+00,4.493916e+01,1.666433,1.078418,0.879144,1.405546e+01,4.346844,-9.087301e+00,0.749151
var_1,0.061300,-0.006694,16.402858,0.042574,0.000085,0.001995,-0.028717,0.011434,0.020920,0.055319,...,2.454047e+00,1.757615e+01,-1.134643e+01,-2.699482,-3.400151,-1.401466,-1.104145e+01,5.670969,-1.962857e+01,-3.044728
var_2,0.044361,0.052770,0.042574,6.974322,0.005401,0.003099,0.032580,0.002020,-0.008942,0.023309,...,-2.231555e+00,7.055100e+00,-1.366476e+02,2.397323,2.786130,-1.227223,-3.927092e+00,-4.391799,2.054248e+01,-0.382044
var_3,0.006792,0.023610,0.000085,0.005401,4.175153,-0.001067,0.052272,-0.001371,0.017460,0.024197,...,-5.051824e+00,2.633418e+00,-1.665438e+01,-4.840652,-0.789245,-4.796072,-4.786308e+00,-2.521134,-1.800505e+01,-0.909636
var_4,0.005326,0.006542,0.001995,0.003099,-0.001067,2.634614,-0.017462,0.000069,0.025237,0.006457,...,-8.411285e-01,-2.418584e+00,1.894165e+00,0.201297,-0.727759,-1.017688,-4.009243e+00,-5.898710,1.428382e+01,2.136457
var_5,0.073237,0.072820,-0.028717,0.032580,0.052272,-0.017462,61.830963,0.017634,-0.026741,0.003851,...,1.023519e+01,-6.422583e+00,-1.546398e+01,15.009411,-0.081967,-5.204095,-6.954774e+01,5.279611,-7.623736e+01,-17.080353
var_6,0.017387,0.018396,0.011434,0.002020,-0.001371,0.000069,0.017634,0.751008,-0.007547,-0.003431,...,-1.506231e+00,-1.432637e+00,-1.612190e+01,0.431553,0.211468,0.299959,1.548310e+00,-1.398274,1.691154e+00,-0.962463
var_7,-0.003108,0.025240,0.020920,-0.008942,0.017460,0.025237,-0.026741,-0.007547,11.683241,0.009277,...,1.126410e+01,-5.137756e-02,1.204114e+02,-7.441729,0.828303,-3.467211,1.730044e+01,-2.214590,1.719830e+01,-1.283620
var_8,0.019623,0.050276,0.055319,0.023309,0.024197,0.006457,0.003851,-0.003431,0.009277,11.106446,...,-2.520381e+01,-1.030611e+01,1.675609e+01,-6.749055,-1.667654,0.142411,3.047903e+01,6.770074,-2.045765e+01,5.145750


In [132]:
X_tophalf, _, y_tophalf, _ = train_test_split(X, y, test_size=0.8, random_state=10)

In [133]:
if USE_STACKING:
    pred_cv = np.zeros(len(X))
    pred = np.zeros(len(X_test))

In [134]:
if USE_STACKING:
    # results from other models
    models = [
        ('lg', SGDClassifier(loss='log', max_iter=5000, tol=1e-7, alpha=0.3)),
        ('mlp', MLPClassifier(solver='lbfgs', alpha=0.001, hidden_layer_sizes=(5, 2), random_state=1)),
        ('rf', RandomForestClassifier(n_estimators=10, criterion='entropy')),
        ('gnb', GaussianNB()),
        ('qda', QuadraticDiscriminantAnalysis(tol=1e-12)),
    ]

    train_models_out = pd.DataFrame(index=train_df.iloc[:,0])
    test_models_out = pd.DataFrame(index=test_df.iloc[:,0])

    for model in models:
        name = model[0]
        cls = model[1]
        print('working on ' + name)

        # k-fold
        n_splits = 5
        folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

        pred_cv = np.zeros(len(X))
        pred = np.zeros(len(X_test))

        for fold_n, (train_index, valid_index) in enumerate(folds.split(X_tophalf,y_tophalf)):
            print("fold: " + str(fold_n))
            X_train, X_valid = X_tophalf.iloc[train_index], X_tophalf.iloc[valid_index]
            y_train, y_valid = y_tophalf.iloc[train_index], y_tophalf.iloc[valid_index]

            cls.fit(X_train, y_train)
            pred_cv += cls.predict_proba(X)[:,1] / n_splits
            pred += cls.predict_proba(X_test)[:,1] / n_splits

        print(pred_cv)

        train_models_out[name] = pred_cv
        test_models_out[name] = pred

In [135]:
if USE_STACKING:
    # add statistics
    num = len(models);
    train_models_out['models_mean'] = train_models_out.iloc[:,[0,num-1]].mean(axis=1)
    train_models_out['models_min'] = train_models_out.iloc[:,[0,num-1]].min(axis=1)
    train_models_out['models_max'] = train_models_out.iloc[:,[0,num-1]].max(axis=1)
    test_models_out['models_mean'] = test_models_out.iloc[:,[0,num-1]].mean(axis=1)
    test_models_out['models_min'] = test_models_out.iloc[:,[0,num-1]].min(axis=1)
    test_models_out['models_max'] = test_models_out.iloc[:,[0,num-1]].max(axis=1)

In [136]:
if USE_STACKING:
    train_models_out.head()

In [137]:
if USE_STACKING:
    test_models_out.head()

In [138]:
if USE_STACKING:
    # add to data
    for col in train_models_out:
        train_df[col] = train_models_out[col].values
        test_df[col] = test_models_out[col].values

In [139]:
if not WITH_ORG_FEATURES:
    for i in range(0, 200):
        train_df = train_df.drop("var_" + str(i), axis=1)
        test_df = test_df.drop("var_" + str(i), axis=1)

In [140]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,mean_dev_std_145_149,mean_dev_std_150_154,mean_dev_std_155_159,mean_dev_std_160_164,mean_dev_std_165_169,mean_dev_std_170_174,mean_dev_std_175_179,mean_dev_std_180_184,mean_dev_std_185_189,mean_dev_std_190_194
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.819083,57.147937,2.092287,0.259931,-1.321391,-1.093963,-1.131768,0.093384,0.603906,3.227312
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,0.184871,0.006106,1.176163,0.071424,-0.215371,12.056620,0.543880,0.225690,-0.985959,0.059058
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,-1.045980,-1.697118,4.508609,-0.356016,3.450665,-0.249342,-1.237436,0.409904,-0.999553,0.623788
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,-2.608053,1.194680,-0.346637,-0.257923,-2.229138,-1.980090,-0.034716,-0.929145,4.669675,0.992789
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,1.655779,0.348970,0.437745,2.072087,0.183004,0.104375,2.574289,-0.870095,-5.874685,-3.859106


In [141]:
test_df.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,mean_dev_std_145_149,mean_dev_std_150_154,mean_dev_std_155_159,mean_dev_std_160_164,mean_dev_std_165_169,mean_dev_std_170_174,mean_dev_std_175_179,mean_dev_std_180_184,mean_dev_std_185_189,mean_dev_std_190_194
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-1.608149,-1.045347,-0.475090,-0.327048,-1.182033,-0.837609,-1.131746,-0.415472,0.112337,-0.890152
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,-4.371639,-1.246999,-0.497493,-0.463703,-1.271714,-0.569389,-0.655675,-0.353988,-0.289971,-0.834923
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-1.856973,-1.210591,-0.488297,-0.421577,-1.219332,-0.923047,-1.132208,-0.323641,-0.362106,-0.927789
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,-1.621730,-0.958179,-0.653155,-0.560528,-1.119150,-0.907030,-0.898232,-0.365489,1.593851,-0.907989
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,-2.053970,-1.150454,-0.394181,-0.308844,-1.147255,-0.670856,-1.001080,-0.426009,-0.331024,-0.957882


In [142]:
train_df.to_csv("input/train_more_features.csv", index=False)
test_df.to_csv("input/test_more_features.csv", index=False)